# What is about ? 

Analysis of METABRIC breast cancer data. In particular survival curves for METABRIC data.
There are several versions of METABRIC data on web and kaggle in particular.
Current notebook uses: 
https://www.kaggle.com/raghadalharbi/breast-cancer-gene-expression-profiles-metabric
Other kaggle datasets, "added"  for visiability of notebook there, but not used.

Not used: 

https://www.kaggle.com/gunesevitan/breast-cancer-metabric

https://www.kaggle.com/alexandervc/breast-cancer-omics-bulk-data


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
f = '/kaggle/input/breast-cancer-gene-expression-profiles-metabric/METABRIC_RNA_Mutation.csv'
df = pd.read_csv(f)
df

In [ ]:
print('Clinical fields:')
print( df.columns[:31] )

In [ ]:
df['overall_survival_months'] 
df['overall_survival'].unique()

In [ ]:
pip install lifelines


In [ ]:
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score
from lifelines import KaplanMeierFitter
from lifelines.utils import concordance_index

import matplotlib.pyplot as plt
import seaborn as sns


# Toy example - Walton Dataset built-in in "lifelines"  package

In [ ]:
from lifelines import KaplanMeierFitter
from lifelines.datasets import load_waltons
waltons = load_waltons()

kmf = KaplanMeierFitter(label="waltons_data")
kmf.fit(waltons['T'], waltons['E'])
kmf.plot()

# Survival curves by Kaplan Meier method

In [ ]:
print( df['overall_survival_months'].isna().sum(), df['overall_survival'].isna().sum() )
T = df['overall_survival_months'] 
E = df['overall_survival']
#T = df['Overall Survival (Months)']
#E =  df['Overall Survival Status'].map({'Living':1, 'Deceased':0} )
kmf = KaplanMeierFitter(label="metabric")
kmf.fit(T,E)
kmf.plot()

In [ ]:
f = 'pam50_+_claudin-low_subtype'
print( df[f].unique() )
df[f].value_counts()

In [ ]:
plt.figure(figsize = (20,15))
for v in ['claudin-low', 'LumA', 'LumB', 'Normal', 'Her2', 'Basal']:
    m = df[f] == v
    T = df['overall_survival_months'][m] 
    E = df['overall_survival'][m]
    #T = df['Overall Survival (Months)'][m]
    #E =  df['Overall Survival Status'][m].map({'Living':1, 'Deceased':0} )
    kmf = KaplanMeierFitter(label=v)
    kmf.fit(T,E)
    kmf.plot()
    

In [ ]:
for c in ['her2_status', '3-gene_classifier_subtype', 'er_status_measured_by_ihc', 'er_status',
       'tumor_other_histologic_subtype', 'hormone_therapy',
       'inferred_menopausal_state', 'integrative_cluster',
       'primary_tumor_laterality', 'lymph_nodes_examined_positive',
       'mutation_count', 'nottingham_prognostic_index', 'oncotree_code',
        'pr_status',
       'radio_therapy',          
         ]:
    plt.figure(figsize = (20,5))
    for v in df[c].unique(): #  ['claudin-low', 'LumA', 'LumB', 'Normal', 'Her2', 'Basal']:
        m = df[c] == v
        if m.sum() < 50: continue 
        T = df['overall_survival_months'][m] 
        E = df['overall_survival'][m]
        #T = df['Overall Survival (Months)'][m]
        #E =  df['Overall Survival Status'][m].map({'Living':1, 'Deceased':0} )
        kmf = KaplanMeierFitter(label=v)
        kmf.fit(T,E)
        kmf.plot()
        plt.title(c)


# Omics data visualizations

In [ ]:
# We see that data are normalized - mean 0, std = 1
stat = df.iloc[:,32:].describe()
list_c = stat.columns
stat

In [ ]:
df.columns[:32]

In [ ]:
df.iloc[:,33:]

In [ ]:
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
pca = PCA
r = pca().fit_transform(X = df[list_c].values)
f = 'pam50_+_claudin-low_subtype' # 'Pam50 + Claudin-low subtype'
plt.figure(figsize = (20,10))
sns.scatterplot( x=r[:,0], y=r[:,1], hue = df[f] )
plt.title('PCA for Omics data colored by Pam50 groups')
plt.show()

In [ ]:
for f in df.columns[:32]:
    plt.figure(figsize = (20,5))
    sns.scatterplot( x=r[:,0], y=r[:,1], hue = df[f] )
    plt.title('PCA for Omics data colored by' + f)
    plt.show()

# Look on categorical features

In [ ]:
cat_clinical_columns = ['chemotherapy', 'cohort', 'neoplasm_histologic_grade','hormone_therapy', 'overall_survival', 'radio_therapy', 'tumor_stage' ]

#cat_clinical_columns.extend(clinical_df.select_dtypes(include=['object']).columns.tolist())
#clinical_df[cat_clinical_columns].astype('category').describe().T

df[cat_clinical_columns].astype('category').describe().T

In [ ]:
#cat_clinical_columns.extend(df.select_dtypes(include=['object']).columns.tolist())
df.select_dtypes(include=['object']).astype('category').describe().T

In [ ]:
df['hras_mut'].value_counts()

In [ ]:
df['smarcb1_mut'].value_counts()

In [ ]:
df['siah1_mut'].value_counts()

In [ ]:
m = df['smarcb1_mut']== 'I28L'
print(m.sum())
df

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

d = df.select_dtypes(include=['object'])[m]
l = []
for c in d.columns:
    if d[c][m].iat[0] != '0':
        l.append(c)
d[l]


In [ ]:
list_c = df.iloc[:1,32:].select_dtypes(include=['object']).columns.to_list() # .astype('category').describe().T

In [ ]:
for c in list_c[:2]:
    print(c)
    print( df[c].value_counts()[:20]  )
    print()